In [2]:
import sys
sys.path.insert(0,r"C:\Users\pc\Documents")
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
## Lectura de datos     
base_oriente = pd.read_csv(r'C:\Users\pc\Downloads\BASE_ORIENTE.txt', sep=r'¡', engine='python')
base_oriente.head()

## Eliminación de variables: 
base_oriente2 = base_oriente[['CUCONUSE', 'SESUSERV', 'SERVDESC', 'SESUSUSC', 'SESUFEIN','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'SUSCNITC', 
        #'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','V1', 'V2', 'V3', 'V4', 'V5', 'V6',#'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 
        'TIPIFICACION_CLIENTE', 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO',
       #'VECTOR_CUALITATIVO_CONTRATO',#'BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION','RANGO_EDAD', 'GENERO','PROM_SUSC', 'CANAL_ENVIO'
       'SEGMENTACION', 'REGIONAL','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

base_oriente3 = base_oriente2.dropna()

In [4]:
## Group by por nit para calificacion
b4 = base_oriente3[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
group_nit = b4.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()


## Join de la nueva calificacion
base_oriente4 = pd.merge(base_oriente3, group_nit, how='left', on=['SUSCNITC'])

## Prueba promedio
b5 = base_oriente4[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y']]

### Creacion de la etiqueta de clasificacion
base_oriente4['y'] = pd.cut(x=base_oriente4['CALIFICACION_VECTOR_SERVICIO_y'],
                            bins=[-1,50,76,100],
                            labels=['No pago','Pago inoportuno','Pago'])


In [5]:
dataTypeSeries = base_oriente4.dtypes
dataTypeSeries

cat = base_oriente4.select_dtypes(include=['object','category'])
cuanti = base_oriente4.select_dtypes(include=['int64','float64'])
  

le = LabelEncoder()
cat = cat.apply(lambda col: le.fit_transform(col))
cat.head(10)

base_oriente5=cuanti.join(cat)
print(base_oriente5.dtypes)

base_oriente5.to_csv(r'C:\Users\pc\Desktop\base_oriente_process.csv')
base_oriente5 = pd.read_csv(r'C:\Users\pc\Desktop\base_oriente_process.csv', sep=',', engine='python')


X = base_oriente5[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 'SESUSUSC','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 
        #'SESUFEIN','SUSCNITC', 'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','RANGO_EDAD', 'GENERO', 
        #'V1', 'V2', 'V3', 'V4', 'V5', 'V6','V7', 'V8', 'V9', 'V10', 'V11', 'V12','REGIONAL','PROM_SUSC', 'CANAL_ENVIO'     
        # 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO_x', 'CALIFICACION_VECTOR_SERVICIO_y',
        #'VECTOR_CUALITATIVO_CONTRATO','BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION', 
       'SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO']]

y = base_oriente5['y']


CUCONUSE                            int64
SESUSERV                            int64
SESUSUSC                            int64
SESUCUSA                            int64
SESUSAPE                            int64
SESUCICL                            int64
SUMA                                int64
CANTIDAD                            int64
CALIFICACION_VECTOR_SERVICIO_x      int64
ESTRATO_AGRUPADO                  float64
ANTIGUEDAD_DIAS                     int64
CALIFICACION_VECTOR_SERVICIO_y    float64
SERVDESC                            int32
SESUFEIN                            int32
DEPADESC                            int32
SUSCNITC                            int32
TIPIFICACION_CLIENTE                int32
SEGMENTACION                        int32
REGIONAL                            int32
DEPARTAMENTO_AGR                    int32
CANAL_PAGO                          int32
y                                   int32
dtype: object


In [6]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5),
    GaussianNB()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

instanteFinal = datetime.now()
tiempo = instanteFinal - instanteInicial # Devuelve un objeto timedelta
segundos = tiempo.seconds

C:\Users\pc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


KNeighborsClassifier
****Results****
Accuracy: 81.8321%
Log Loss: 2.774661574532393
DecisionTreeClassifier
****Results****
Accuracy: 78.7923%
Log Loss: 0.48282138454684226
RandomForestClassifier
****Results****
Accuracy: 68.3152%
Log Loss: 0.6556393036124466
MLPClassifier
****Results****
Accuracy: 81.0123%
Log Loss: 0.4714446723951566
LogisticRegression
****Results****
Accuracy: 73.9127%
Log Loss: 0.5811067296802601
GaussianNB
****Results****
Accuracy: 73.3948%
Log Loss: 1.1681431070880992


In [ ]:
## El mejor modelo de clasificacion fue el KNN

In [8]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
for train_index, test_index in kf.split(X, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    #model = MLPClassifier(alpha=1, max_iter=500).fit(X_train, y_train) 
    model = KNeighborsClassifier(3).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                       y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

[             precision    recall  f1-score  support     pred       AUC
0             0.652275  0.706769  0.678430  10282.0  11141.0  0.867743
1             0.788431  0.783413  0.785914  30325.0  30132.0  0.823181
2             0.542298  0.518819  0.530299  15383.0  14717.0  0.752174
avg / total   0.695803  0.696642  0.695947  55990.0  55990.0  0.837875,              precision    recall  f1-score  support     pred       AUC
0             0.621398  0.599883  0.610451  10282.0   9926.0  0.799574
1             0.758153  0.782754  0.770257  30325.0  31309.0  0.788384
2             0.546089  0.523794  0.534709  15382.0  14754.0  0.751716
avg / total   0.674778  0.678026  0.676197  55989.0  55989.0  0.816965,              precision    recall  f1-score  support     pred       AUC
0             0.683745  0.691791  0.687745  10282.0  10403.0  0.862456
1             0.747352  0.779489  0.763082  30325.0  31629.0  0.809934
2             0.494161  0.448381  0.470159  15382.0  13957.0  0.748161
avg